Here, we are going to parse the resume that user attaches

In [11]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.output_parsers import JsonOutputParser
from langchain.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
import ollama

In [10]:
text_to_parse =  'Priyanka Pandey  \nprnkpandey00@gmail.com | +91 7895482254  \nhttps://www.linkedin.com/in/priyankapandey03/ | https://github.com/prnk04 \n  \nSUMMARY  \nSoftware Engineer with 7+ years of experience, transitioning into ML/Data Science. Skilled in recommendation \nsystems, NLP embeddings, and end-to-end ML apps. Completed MIT Applied Data Science Program and built \nEchoFinder, a music recommendation platform. Actively seeking ML/Data Science opportunities. \nSKILLS \nML & Statistics  Classification, Regression, Clustering, Recommendation Systems, Time Series Analysis, NLP, \nStatistical Analysis \nProgramming  Python (NumPy, Pandas, Scikit-learn), FastAPI, SQL, JavaScript, React Native, Express.js, \nNode.js, Redux  \nData Engineering  ETL Pipelines, REST API, MongoDB, JSON/XML Data Processing, Data Preprocessing & \nCleaning  \nData Visualization Matplotlib, Seaborn  \nCloud & Deployment  AWS (EC2, S3, Rekognition), Google Firebase (Cloud Messaging, Analytics, Maps), Google \nPlay Console, Apple App Store Connect \nTools & Platforms Git, GitHub, Splunk, Postman, SoapUI, JIRA, VS Code, Jupyter Notebook \nEXPERIENCE \nIsodate Inc. | Lead Software Engineer (Remote) Aug 2022 - May 2024 \n• Built and shipped cross-platform mobile app with 5.7K+ downloads, using React Native  \n• Integrated AWS EC2, S3, additionally Rekognition to enable content moderation  \n• Implemented Firebase Cloud Messaging, Analytics, & Crashlytics \n• Delivered live event support, reducing user-reported issues and enabling smooth event execution \n• Collaborated with external partners (Lu.Ma, Vonage, CometChat) to scale platform capabilities  \nInfosys Limited | Technology Analyst (Pune, India) Aug 2021 - Jul 2022 \n• Fine-tuned GPT-J to build an ML-based Test Script Generator, converting natural language to Selenium scripts \nwith 95% accuracy, reducing manual QA workload  \n• Developed a Python + ANTLR transpiler converting VB6 to Java, accelerating legacy modernization initiatives  \nInfosys Limited | Senior Associate Consultant (Pune, India) Mar 2019 - Jul 2021 \n• Led offshore integration development for a US healthcare client using IBM Middleware, ensuring high-volume \nclaim processing reliability  \n• Built Splunk dashboards and automated alerts to identify system bottlenecks, improving visibility and reducing \ndebugging time  \n• Received Insta Award for exceptional performance and delivery quality \nIBM India Pvt. Ltd. | Associate System Engineer (Pune, India) Oct 2016 - Feb 2019 \n• Developed 17 enterprise integration applications for retail modernization  \n• Built analytics dashboards for resource utilization and workforce planning, adopted by leadership  \n• Earned two Manager Choice Awards for high-visibility data visualization tools \nPROJECTS \nEchoFinder - AI Music Recommendation System (https://github.com/prnk04/EchoFinder) \n• Built content-based recommendation engine using semantic embeddings (all-MiniLM-L6-v2) and cosine \nsimilarity across 30K+ tracks  \n• Developed FastAPI service exposing ML inference endpoints for real-time recommendations  \n• Implemented an Express.js service layer managing song retrieval, and playlist operations \n• Built a React Native mobile interface providing a clean, intuitive UI receiving personalized recommendations. \nMusic Recommendation System - MIT Capstone Project  \n• Built collaborative filtering models, SVD Matrix Factorization, and Clustering-based recommenders  \n• Evaluated models using precision@K, recall@K, and F1 score. \n• Tuned hyperparameters to improve recommendation accuracy  \n• Applied feature engineering on user-item interactions for improved personalization  \n• Implemented using Python, Pandas, Scikit-learn, Surprise  \nPulsePlay - React Native Music Player (https://github.com/prnk04/PulsePlay) \n• Developed a cross-platform music player with playlists, categories, and smooth playback  \n• Implemented using React Native and Redux Toolkit for state management  \n \nEDUCATION & CERTIFICATES \nMIT Applied Data Science Program  \nSkills: Machine Learning, Statistics, Deep Learning, Neural Networks, Python Programming \nSep 2024 - Dec 2024 \nB.Tech - Computer Science & Engineering  \nBTKIT Dwarahat, Uttarakhand \n2012-2016 \nAWS Certified Cloud Practitioner Dec 2021'

We want the LLM to:
- remove PII like conact no, address
- extract:
- skills
- experience in the form of a list with start data, end date: infer duration
- education
- certificates
- projects

In [12]:
MODEL_NAME = "qwen3:8b"
llm = ChatOllama(model=MODEL_NAME)

In [20]:
resume_parsing_agent = create_agent(
    # model="qwen3:8b",
    model=llm,
    system_prompt=SystemMessage(
        content = [
            {
                "type": "text",
                "text": "You are an assistant at an organisation that actively reviews and evaluates candidates' resume. You perform the tasks assign to you flawlessly"

            }
        ]
    )
)

agent_res = resume_parsing_agent.invoke({
    "messages": [
        HumanMessage(
            content=f"""
                        You are an expert in reviewing, and analysing resume. You are given contents of a resume in text format.Your task includes:
                            - reviewing the resume contents
                            - notifying about the presence of any PII(Personally Identifiable Information), and categorise them
                            - extract following information from the resume:
                                - summary section from the resume. If user has provided summary, or objective, extract that
                                - skills
                                - experience: Extract list of experience of the format: 
                                    {{
                                        "company:"",
                                        "position": "",
                                        "start_date":"",
                                        "end_date":"",
                                        "duration": <calculate this from end date and start date>,
                                        "is_this_current_role":"True/False",
                                        "role_and_responsibilities":<list_of_roles_and_responsibilities"
                                    }}
                                - education: Extract list of education provided
                                - projects: Extract list of personal projects, along with their names, links(if provided), summary.
                                - Certifications
                        The final JSON data that you should return, should be follow the following schema:
                        {{
                            'name':str,
                            'pii': list(),
                            'summary':str,
                            'skills': list(),
                            'experience': list(),
                            'education: list(),
                            'projects': list(),
                            'certifications': list()
                        }}
                        Base your result on the provided information only. Always return valud JSON.

                        resume_text: {text_to_parse}
                        
                    """
        )
    ]
})

In [22]:
agent_res.get("messages")[-1].content

'{\n    "name": "Priyanka Pandey",\n    "pii": [\n        {\n            "category": "Email",\n            "value": "prnkpandey00@gmail.com"\n        },\n        {\n            "category": "Phone Number",\n            "value": "+91 7895482254"\n        }\n    ],\n    "summary": "Software Engineer with 7+ years of experience, transitioning into ML/Data Science. Skilled in recommendation systems, NLP embeddings, and end-to-end ML apps. Completed MIT Applied Data Science Program and built EchoFinder, a music recommendation platform. Actively seeking ML/Data Science opportunities.",\n    "skills": [\n        "Classification",\n        "Regression",\n        "Clustering",\n        "Recommendation Systems",\n        "Time Series Analysis",\n        "NLP",\n        "Statistical Analysis",\n        "Python (NumPy, Pandas, Scikit-learn)",\n        "FastAPI",\n        "SQL",\n        "JavaScript",\n        "React Native",\n        "Express.js",\n        "Node.js",\n        "Redux",\n        "ETL

In [21]:
import json

In [24]:
formatted_data = json.loads(agent_res.get("messages")[-1].content)
formatted_data

{'name': 'Priyanka Pandey',
 'pii': [{'category': 'Email', 'value': 'prnkpandey00@gmail.com'},
  {'category': 'Phone Number', 'value': '+91 7895482254'}],
 'summary': 'Software Engineer with 7+ years of experience, transitioning into ML/Data Science. Skilled in recommendation systems, NLP embeddings, and end-to-end ML apps. Completed MIT Applied Data Science Program and built EchoFinder, a music recommendation platform. Actively seeking ML/Data Science opportunities.',
 'skills': ['Classification',
  'Regression',
  'Clustering',
  'Recommendation Systems',
  'Time Series Analysis',
  'NLP',
  'Statistical Analysis',
  'Python (NumPy, Pandas, Scikit-learn)',
  'FastAPI',
  'SQL',
  'JavaScript',
  'React Native',
  'Express.js',
  'Node.js',
  'Redux',
  'ETL Pipelines',
  'REST API',
  'MongoDB',
  'JSON/XML Data Processing',
  'Data Preprocessing & Cleaning',
  'Matplotlib',
  'Seaborn',
  'AWS (EC2, S3, Rekognition)',
  'Google Firebase (Cloud Messaging, Analytics, Maps)',
  'Google

In [36]:
job_title_agent = create_agent(
    # model="qwen3:8b",
    model=llm,
    system_prompt=SystemMessage(
        content = [
            {
                "type": "text",
                "text": """You are a skeptical hiring manager who has been burned by inflated resumes. 
                            You only give high scores (>0.85) when evidence is overwhelming and undeniable.
                            Your goal is to conservatively assess which job titles a candidate is realistically suitable for based only on evidence in the resume. "
                            You prefer under-recommending over over-recommending."""

            }
        ]
    )
)

title_matching_agent_res = resume_parsing_agent.invoke({
    "messages": [
        HumanMessage(
            content=f"""
                        You are an expert in reviewing, and analysing resume. You are given contents of a resume in JSON format.Your task includes:
                            - reviewing the resume contents
                            - create user profile based on the resume contents
                            - finding out job titles that best match user profile
                            - For each job title:
                                1. First, decide if the candidate is eligible(yes/no):
                                    - If no, do not include the title
                                2. If yes, compute a match score
                            Match score calibration rules:
                                - 0.90-1.00 → Strong match, candidate meets almost all expectations for this role
                                - 0.75-0.89 → Good match, some gaps but role is realistic
                                - 0.60-0.74 → Partial match, role may require upskilling
                                - <0.60 → Do NOT include
                            Penalise scores if:
                                - No direct job title match in experience
                                - Missing 3+ key skills for role
                                - Less than 2 years in related work
                                - Fresher/no experience, unless perfect skill match
                            Do NOT give scores above 0.9 unless the resume clearly demonstrates direct experience in that role.

                            - return top-5 such titles, along with:
                                - the confidence score, on a scale of 0.0-1.0
                                - reason as to why those titles match users profile. 
                            Each reason MUST include:
                                - 1-2 strengths that support the match
                                - 1 explicit gap or limitation relevant to that role

                            If the resume does not provide enough evidence to confidently recommend a role, you MUST exclude it.
                                
                            
                        The final JSON data that you should return, should be follow the following schema:
                        {[{
                            'job_title': str,
                            'match_score': float,
                            'reason': str
                        }]}
                        Base your result on the provided information only. Always return valud JSON.

                        resume_text: {formatted_data}
                        
                    """
        )
    ]
})

In [37]:
formatted_job_titles = json.loads(title_matching_agent_res.get("messages")[-1].content)
formatted_job_titles

[{'job_title': 'Machine Learning Engineer',
  'match_score': 0.9,
  'reason': 'Strengths: Direct experience in building recommendation systems and NLP embeddings, along with AWS cloud integration. Gap: Limited explicit mention of ML engineering frameworks (e.g., TensorFlow/PyTorch) in past roles.'},
 {'job_title': 'Data Scientist',
  'match_score': 0.85,
  'reason': 'Strengths: MIT Applied Data Science certification and hands-on experience with ML models (SVD, clustering). Gap: Fewer examples of data science-specific tools (e.g., Spark, Hadoop) in the resume.'},
 {'job_title': 'NLP Engineer',
  'match_score': 0.8,
  'reason': 'Strengths: Expertise in semantic embeddings and NLP-based recommendation systems. Gap: Minimal direct experience with NLP pipeline optimization or transformer-based architectures.'},
 {'job_title': 'Full-Stack Developer (ML Focus)',
  'match_score': 0.75,
  'reason': 'Strengths: Cross-platform app development (React Native) and ML inference deployment (FastAPI). 

In [41]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://www.adzuna.in/land/ad/5608595165?se=LDky4VoC8RGgSulax50f_Q&utm_medium=api&utm_source=70a709d9&v=F241ABF4EA6616E5A37F4DFCFAFE2C559328C353&app_id=70a709d9&app_key=2cf1ec5a8b2a059a004c25c0a5e3c044",
                       )